In [2]:
import sys
sys.path.append("../../")  
import numpy as np
import pandas as pd
from tqdm import tqdm
from sl_inference_varied_source.mock_generator.lens_model import LensModel
from sl_inference_varied_source.mock_generator.lens_properties import lens_properties
# from ..mock_generator.mass_sampler import MODEL_PARAMS, sample_m_s
from sl_inference_varied_source.mock_generator.mass_sampler import sample_m_s
# === 参数设定 ===
n_samples = 10000
seed = 42
mu_DM = 13.0
sigma_DM = 0.2
n_sigma = 3
alpha_s = -1.3
m_s_star = 24.5

zl, zs = 0.3, 2.0
maximum_magnitude = 26.5  # 检出限制
OBS_SCATTER_MAG = 0.1     # 源星等噪声
OBS_SCATTER_STAR = 0.1    # logM_sps 噪声

# === 随机采样五维参数空间 ===
rng = np.random.default_rng(seed)
logMstar = rng.normal(11.4, 0.3, n_samples)
logRe = rng.normal(1 + 0.8 * (logMstar - 11.4), 0.15, n_samples)
Mh_min = mu_DM - n_sigma * sigma_DM
Mh_max = mu_DM + n_sigma * sigma_DM
logMh = rng.uniform(Mh_min, Mh_max, n_samples)
beta = rng.uniform(0.0, 1.0, n_samples)
m_s = sample_m_s(alpha_s, m_s_star, size=n_samples, rng=rng)

# === 遍历样本，判断是否为 lens ===
num_lensed = 0

for i in tqdm(range(n_samples)):
    model = LensModel(
        logM_star=logMstar[i],
        logM_halo=logMh[i],
        logRe=logRe[i],
        zl=zl,
        zs=zs
    )
    props = lens_properties(model, beta[i])

    muA, muB = props['magnificationA'], props['magnificationB']
    magA = m_s[i] - 2.5 * np.log10(muA) + rng.normal(0.0, OBS_SCATTER_MAG)
    magB = m_s[i] - 2.5 * np.log10(muB) + rng.normal(0.0, OBS_SCATTER_MAG)

    if magA < maximum_magnitude and magB < maximum_magnitude:
        num_lensed += 1

# === 输出结果 ===
frac_lensed = num_lensed / n_samples
print(f"[INFO] 可观测透镜比例：{frac_lensed:.4f}（共 {n_samples} 个样本中 {num_lensed} 个）")


100%|██████████| 10000/10000 [00:33<00:00, 302.73it/s]

[INFO] 可观测透镜比例：0.3628（共 10000 个样本中 3628 个）
